In [ ]:
#default_exp data.blocks

In [ ]:
from fastai_local.imports import *
from fastai_local.test import *
from fastai_local.core import *
from fastai_local.data.pipeline import *
from fastai_local.data.source import *
from fastai_local.data.core import *
from fastai.datasets import *

# Data blocks

## TfmList -

In [ ]:
#export
class TfmList():
    def __init__(self, tfms): self.activ,self.tfms = False,[Pipeline(t) for t in listify(tfms)]

    def __call__(self, o, **kwargs):
        if self.activ: return self.activ(o, **kwargs)
        return [t(o, **kwargs) for t in self.tfms]
    
    def decode(self, o, **kwargs): return [t.decode(p, **kwargs) for p,t in zip(o,self.tfms)]

    def setup(self, o):
        for tfm in self.tfms:
            self.activ = tfm
            tfm.setup(o)
        self.activ=None
        
    def show(self, o, **kwargs): return show_xs(o, self.tfms, **kwargs)
    def __repr__(self): return f'TfmList({self.tfms})'
    
    @property
    def xt(self): return self.tfms[0]
    @property
    def yt(self): return self.tfms[1]

In [ ]:
tadd  = Transform(lambda x: x+1, decodes=lambda x: x-1 , order=2)
tmult = Transform(lambda x: x*2, decodes=lambda x: x//2, order=1)

tfm = TfmList([tadd,tmult])
test_eq(tfm(3), (4, 6))

In [ ]:
class TNorm_(Transform):
    def __init__(self): self.m,self.s = 0,1
    def encodes(self, o): return (o-self.m)/self.s
    def decodes(self, o): return (o*self.s)+self.m
    def setup(self, dsrc):
        its = tensor(dsrc.train)
        self.m,self.s = its.mean(),its.std()

tnrm = TNorm_()
tfm = TfmList([tadd,tnrm])
source = [1,2,3,4]
dsrc = DataSource(source, tfm)
x,y = zip(*dsrc.train)
test_close(tensor(y).mean(), 0)
test_close(tensor(y).std(), 1)
test_eq(x, (2,3,4,5))